# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,972.938987,0.209500,1234,973.148488,972.729487,1946.296975,1945.458974
6,641.811107,0.364714,1234,642.175821,641.446393,1284.351642,1282.892786
7,516.145170,-0.763685,1234,515.381486,516.908855,1030.762971,1033.817709
8,474.276310,0.331875,1234,474.608185,473.944434,949.216370,947.888869
9,582.850520,0.835577,1234,583.686097,582.014943,1167.372195,1164.029886
10,162.058104,-0.787759,1234,161.270344,162.845863,322.540689,325.691727
11,353.151974,0.265310,1234,353.417284,352.886664,706.834567,705.773327
12,503.446974,0.069297,1234,503.516272,503.377677,1007.032543,1006.755353
13,838.874276,0.934608,1234,839.808883,837.939668,1679.617767,1675.879336
14,284.951188,-0.850740,1234,284.100448,285.801929,568.200897,571.603857


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-973.148488,972.729487
6,-642.175821,641.446393
7,-515.381486,516.908855
8,-474.608185,473.944434
9,-583.686097,582.014943
10,-161.270344,162.845863
11,-353.417284,352.886664
12,-503.516272,503.377677
13,-839.808883,837.939668
14,-284.100448,285.801929


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-973.148488,972.729487
6,-642.175821,641.446393
7,-515.381486,516.908855
8,-474.608185,473.944434
9,-583.686097,582.014943
10,-161.270344,162.845863
11,-353.417284,352.886664
12,-503.516272,503.377677
13,-839.808883,837.939668
14,-284.100448,285.801929


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-973.1484875 ,  972.72948691],
       [-642.17582103,  641.44639283],
       [-515.38148555,  516.9088546 ],
       [-474.60818486,  473.94443441],
       [-583.6860973 ,  582.01494324],
       [-161.27034432,  162.8458633 ],
       [-353.41728366,  352.88666369],
       [-503.51627153,  503.37767668],
       [-839.80888342,  837.93966812],
       [-284.10044843,  285.8019285 ]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1944.296975,-1945.458974
6,1282.351642,-1282.892786
7,1028.762971,-1033.817709
8,947.216370,-947.888869
9,1165.372195,-1164.029886
10,320.540689,-325.691727
11,704.834567,-705.773327
12,1005.032543,-1006.755353
13,1677.617767,-1675.879336
14,566.200897,-571.603857


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [16]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [17]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [18]:
data['a']  = np.nan

In [19]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [20]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [21]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
